<h1 align = "center"> Spark DataFrames </h1>

Ověření inicializace Spark contextu.

In [1]:
sc.version

2.4.3

Vytvoření SparkSession.

In [4]:
 val spark = SparkSession
      .builder
      .appName("SparkSQL")
      .master("local[*]")
      .getOrCreate()

spark = org.apache.spark.sql.SparkSession@3bf57885


org.apache.spark.sql.SparkSession@3bf57885

# Vytvoření DataFramu

Základní nastavení

In [5]:
 import spark.implicits._

In [6]:
case class Weather(date: String, temp: Int, precipitation: Double)

defined class Weather


Vytvoření dataframu z RDD

In [7]:
val data = sc.textFile("LabData/nycweather.csv")
val header = data.first()
val weatherFromRdd = data.filter(row => row != header).map(_.split(",")).map(w => Weather(w(0), w(1).trim.toInt, w(2).trim.toDouble)).toDF()

data = LabData/nycweather.csv MapPartitionsRDD[1] at textFile at <console>:34
header = date,temp,precipitation
weatherFromRdd = [date: string, temp: int ... 1 more field]


[date: string, temp: int ... 1 more field]

Vytvoření DataFramu.

In [8]:
val weatherDataFrame = spark.read.option("header", true).option("inferSchema", true).csv("LabData/nycweather.csv")

weatherDataFrame = [date: timestamp, temp: int ... 1 more field]


[date: timestamp, temp: int ... 1 more field]

Vytvoření Datasetu.

In [9]:
val weather = spark.read.option("header", true).option("inferSchema", true)
      .csv("LabData/nycweather.csv")
      .as[Weather]

weather = [date: timestamp, temp: int ... 1 more field]


[date: timestamp, temp: int ... 1 more field]

Porovnání rozdílu DataFramu a Datasetu

In [10]:
 weather.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- temp: integer (nullable = true)
 |-- precipitation: double (nullable = true)



In [11]:
weatherDataFrame.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- temp: integer (nullable = true)
 |-- precipitation: double (nullable = true)



In [ ]:
weatherFromRdd.printSchema()

root
 |-- date: string (nullable = true)
 |-- temp: integer (nullable = false)
 |-- precipitation: double (nullable = false)



In [13]:
printf(weatherFromRdd.first().getClass.getSimpleName)

GenericRowWithSchema

In [14]:
printf(weather.first().getClass.getSimpleName)

Weather

## Práce s DataFramem

In [27]:
weather.show()

+----------+----+-------------+
|      date|temp|precipitation|
+----------+----+-------------+
|2013-01-01|   1|          0.0|
|2013-01-02|  -2|          0.0|
|2013-01-03|  -2|          0.0|
|2013-01-04|   1|          0.0|
|2013-01-05|   3|          0.0|
|2013-01-06|   4|          0.0|
|2013-01-07|   5|          0.0|
|2013-01-08|   6|          0.0|
|2013-01-09|   7|          0.0|
|2013-01-10|   7|          0.0|
|2013-01-11|   6|        13.97|
|2013-01-12|   7|         0.51|
|2013-01-13|   8|          0.0|
|2013-01-14|   8|         2.29|
|2013-01-15|   3|         3.05|
|2013-01-16|   2|        17.53|
|2013-01-17|   4|          0.0|
|2013-01-18|  -1|          0.0|
|2013-01-19|   5|          0.0|
|2013-01-20|   6|          0.0|
+----------+----+-------------+
only showing top 20 rows



### SELECT

In [28]:
weather.select("date").show()

+-------------------+
|               date|
+-------------------+
|2013-01-01 00:00:00|
|2013-01-02 00:00:00|
|2013-01-03 00:00:00|
|2013-01-04 00:00:00|
|2013-01-05 00:00:00|
|2013-01-06 00:00:00|
|2013-01-07 00:00:00|
|2013-01-08 00:00:00|
|2013-01-09 00:00:00|
|2013-01-10 00:00:00|
|2013-01-11 00:00:00|
|2013-01-12 00:00:00|
|2013-01-13 00:00:00|
|2013-01-14 00:00:00|
|2013-01-15 00:00:00|
|2013-01-16 00:00:00|
|2013-01-17 00:00:00|
|2013-01-18 00:00:00|
|2013-01-19 00:00:00|
|2013-01-20 00:00:00|
+-------------------+
only showing top 20 rows



In [30]:
weather.createOrReplaceTempView("weather")
spark.sql("SELECT date FROM weather").show()

+-------------------+
|               date|
+-------------------+
|2013-01-01 00:00:00|
|2013-01-02 00:00:00|
|2013-01-03 00:00:00|
|2013-01-04 00:00:00|
|2013-01-05 00:00:00|
|2013-01-06 00:00:00|
|2013-01-07 00:00:00|
|2013-01-08 00:00:00|
|2013-01-09 00:00:00|
|2013-01-10 00:00:00|
|2013-01-11 00:00:00|
|2013-01-12 00:00:00|
|2013-01-13 00:00:00|
|2013-01-14 00:00:00|
|2013-01-15 00:00:00|
|2013-01-16 00:00:00|
|2013-01-17 00:00:00|
|2013-01-18 00:00:00|
|2013-01-19 00:00:00|
|2013-01-20 00:00:00|
+-------------------+
only showing top 20 rows



### WHERE

In [33]:
weather.select("*").where(weather("temp") > 5).show()

+-------------------+----+-------------+
|               date|temp|precipitation|
+-------------------+----+-------------+
|2013-01-08 00:00:00|   6|          0.0|
|2013-01-09 00:00:00|   7|          0.0|
|2013-01-10 00:00:00|   7|          0.0|
|2013-01-11 00:00:00|   6|        13.97|
|2013-01-12 00:00:00|   7|         0.51|
|2013-01-13 00:00:00|   8|          0.0|
|2013-01-14 00:00:00|   8|         2.29|
|2013-01-20 00:00:00|   6|          0.0|
|2013-01-29 00:00:00|   6|         1.52|
|2013-01-30 00:00:00|   9|         1.02|
|2013-01-31 00:00:00|   8|        22.86|
|2013-02-15 00:00:00|   8|          0.0|
|2013-02-27 00:00:00|   6|        39.62|
|2013-02-28 00:00:00|   7|          0.0|
|2013-03-09 00:00:00|   7|          0.0|
|2013-03-10 00:00:00|   6|          0.0|
|2013-03-11 00:00:00|   8|          0.0|
|2013-03-12 00:00:00|  10|        20.07|
|2013-03-13 00:00:00|   7|          0.0|
|2013-03-26 00:00:00|   7|          0.0|
+-------------------+----+-------------+
only showing top

In [34]:
spark.sql("SELECT * FROM weather WHERE temp > 5").show()

+-------------------+----+-------------+
|               date|temp|precipitation|
+-------------------+----+-------------+
|2013-01-08 00:00:00|   6|          0.0|
|2013-01-09 00:00:00|   7|          0.0|
|2013-01-10 00:00:00|   7|          0.0|
|2013-01-11 00:00:00|   6|        13.97|
|2013-01-12 00:00:00|   7|         0.51|
|2013-01-13 00:00:00|   8|          0.0|
|2013-01-14 00:00:00|   8|         2.29|
|2013-01-20 00:00:00|   6|          0.0|
|2013-01-29 00:00:00|   6|         1.52|
|2013-01-30 00:00:00|   9|         1.02|
|2013-01-31 00:00:00|   8|        22.86|
|2013-02-15 00:00:00|   8|          0.0|
|2013-02-27 00:00:00|   6|        39.62|
|2013-02-28 00:00:00|   7|          0.0|
|2013-03-09 00:00:00|   7|          0.0|
|2013-03-10 00:00:00|   6|          0.0|
|2013-03-11 00:00:00|   8|          0.0|
|2013-03-12 00:00:00|  10|        20.07|
|2013-03-13 00:00:00|   7|          0.0|
|2013-03-26 00:00:00|   7|          0.0|
+-------------------+----+-------------+
only showing top

### ORDER BY

In [51]:
import org.apache.spark.sql.functions.desc
weather.select("*").orderBy(desc("temp")).show()

+-------------------+----+-------------+
|               date|temp|precipitation|
+-------------------+----+-------------+
|2013-07-18 00:00:00|  32|          0.0|
|2013-07-19 00:00:00|  32|          0.0|
|2013-07-17 00:00:00|  31|          0.0|
|2013-07-20 00:00:00|  31|          0.0|
|2013-09-11 00:00:00|  31|          0.0|
|2013-07-16 00:00:00|  30|          0.0|
|2013-07-15 00:00:00|  30|          0.0|
|2013-07-06 00:00:00|  29|          0.0|
|2013-07-07 00:00:00|  29|          0.0|
|2013-06-24 00:00:00|  28|          0.0|
|2013-06-25 00:00:00|  28|          0.0|
|2013-07-05 00:00:00|  28|          0.0|
|2013-07-21 00:00:00|  28|          0.0|
|2013-05-31 00:00:00|  28|          0.0|
|2013-07-14 00:00:00|  28|          0.0|
|2013-06-01 00:00:00|  28|          0.0|
|2013-05-30 00:00:00|  27|          0.0|
|2013-06-27 00:00:00|  27|          6.1|
|2013-07-09 00:00:00|  27|         5.84|
|2013-06-26 00:00:00|  27|         1.27|
+-------------------+----+-------------+
only showing top

In [56]:
spark.sql("SELECT * FROM weather ORDER BY temp DESC").show()

+-------------------+----+-------------+
|               date|temp|precipitation|
+-------------------+----+-------------+
|2013-07-18 00:00:00|  32|          0.0|
|2013-07-19 00:00:00|  32|          0.0|
|2013-07-17 00:00:00|  31|          0.0|
|2013-07-20 00:00:00|  31|          0.0|
|2013-09-11 00:00:00|  31|          0.0|
|2013-07-16 00:00:00|  30|          0.0|
|2013-07-15 00:00:00|  30|          0.0|
|2013-07-06 00:00:00|  29|          0.0|
|2013-07-07 00:00:00|  29|          0.0|
|2013-06-24 00:00:00|  28|          0.0|
|2013-06-25 00:00:00|  28|          0.0|
|2013-07-05 00:00:00|  28|          0.0|
|2013-07-21 00:00:00|  28|          0.0|
|2013-05-31 00:00:00|  28|          0.0|
|2013-07-14 00:00:00|  28|          0.0|
|2013-06-01 00:00:00|  28|          0.0|
|2013-05-30 00:00:00|  27|          0.0|
|2013-06-27 00:00:00|  27|          6.1|
|2013-07-09 00:00:00|  27|         5.84|
|2013-06-26 00:00:00|  27|         1.27|
+-------------------+----+-------------+
only showing top

### GROUP BY

In [52]:
import org.apache.spark.sql.functions.desc
weather.select("*").groupBy("temp").count().sort(desc("temp")).show()

+----+-----+
|temp|count|
+----+-----+
|  32|    2|
|  31|    3|
|  30|    2|
|  29|    2|
|  28|    7|
|  27|   15|
|  26|   11|
|  25|    7|
|  24|   15|
|  23|   20|
|  22|   13|
|  21|   10|
|  20|    5|
|  19|   12|
|  18|   11|
|  17|   18|
|  16|   14|
|  15|    5|
|  14|   14|
|  13|    8|
+----+-----+
only showing top 20 rows



In [58]:
spark.sql("SELECT temp, count(temp) as count FROM weather GROUP BY temp ORDER BY temp DESC").show()

+----+-----+
|temp|count|
+----+-----+
|  32|    2|
|  31|    3|
|  30|    2|
|  29|    2|
|  28|    7|
|  27|   15|
|  26|   11|
|  25|    7|
|  24|   15|
|  23|   20|
|  22|   13|
|  21|   10|
|  20|    5|
|  19|   12|
|  18|   11|
|  17|   18|
|  16|   14|
|  15|    5|
|  14|   14|
|  13|    8|
+----+-----+
only showing top 20 rows



# Samostatná práce

In [61]:
val spark = SparkSession
      .builder
      .appName("SparkSQL")
      .master("local[*]")
      .getOrCreate()

Name: Unknown Error
Message: <console>:13: error: not found: value SparkSession
       val spark = SparkSession
                   ^

StackTrace: 

In [63]:
case class FakeFriends(id: Int, name: String, age: Int, friends: Long)

defined class FakeFriends


Vytvoření nového Datasetu z souboru fakefriends.csv.

In [67]:
val friends = spark.read
      .option("header", "true")
      .option("inferSchema", "true")
      .csv("LabData/fakefriends.csv")
      .as[FakeFriends]

friends = [id: int, name: string ... 2 more fields]


[id: int, name: string ... 2 more fields]

Vypsat schéma nového Datasetu.  

In [69]:
friends.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- friends: integer (nullable = true)



Vypsání dat v Datasetu.

In [70]:
friends.show()

+---+--------+---+-------+
| id|    name|age|friends|
+---+--------+---+-------+
|  0|    Will| 33|    385|
|  1|Jean-Luc| 26|      2|
|  2|    Hugh| 55|    221|
|  3|  Deanna| 40|    465|
|  4|   Quark| 68|     21|
|  5|  Weyoun| 59|    318|
|  6|  Gowron| 37|    220|
|  7|    Will| 54|    307|
|  8|  Jadzia| 38|    380|
|  9|    Hugh| 27|    181|
| 10|     Odo| 53|    191|
| 11|     Ben| 57|    372|
| 12|   Keiko| 54|    253|
| 13|Jean-Luc| 56|    444|
| 14|    Hugh| 43|     49|
| 15|     Rom| 36|     49|
| 16|  Weyoun| 22|    323|
| 17|     Odo| 35|     13|
| 18|Jean-Luc| 45|    455|
| 19|  Geordi| 60|    246|
+---+--------+---+-------+
only showing top 20 rows



Vybrat uživatele starší 30 let.

In [71]:
val olderUsers = friends.select("*").where(friends("age") > 30)

olderUsers = [id: int, name: string ... 2 more fields]


[id: int, name: string ... 2 more fields]

Seřadit podle věku a sestupně vypsat výsledek.

In [77]:
olderUsers.orderBy(desc("age")).show()

+---+--------+---+-------+
| id|    name|age|friends|
+---+--------+---+-------+
|205|    Morn| 69|    236|
|116|     Ben| 69|     75|
|253|   Leeta| 69|    116|
| 97|   Nerys| 69|    361|
|354|  Kasidy| 69|     15|
|383|    Data| 69|    148|
| 99|   Keiko| 69|    491|
|397|   Quark| 69|    470|
|487|   Brunt| 69|    431|
| 62|   Keiko| 69|      9|
|120|Jean-Luc| 68|    264|
|168|  Martok| 68|    112|
|310|     Odo| 68|    189|
|258|    Worf| 68|    217|
|170|Jean-Luc| 68|    490|
|  4|   Quark| 68|     21|
|361|    Worf| 68|    206|
|428| Lwaxana| 68|    423|
|232|    Worf| 68|    481|
|374|     Nog| 68|    293|
+---+--------+---+-------+
only showing top 20 rows

